In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn.utils import class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
# datasets originais https://www.kaggle.com/datasets/selfishgene/historical-hourly-weather-data?select=weather_description.csv

In [16]:
# dataset = pd.read_csv('dados_originais.csv').drop(['Unnamed: 0', 'id', 'date', 'year', 'day', 'time'], axis = 1)
# dataset

,month,humidity,pressure,temperature,wind,weather
0,10,58,1012,15,7,few_clouds
1,10,57,1012,15,7,few_clouds
2,10,57,1012,15,7,few_clouds
3,10,57,1012,15,7,few_clouds
4,10,57,1012,15,6,few_clouds
...,...,...,...,...,...,...
43575,10,36,1019,16,3,sky_is_clear
43576,10,38,1019,16,1,sky_is_clear
43577,10,54,1019,14,2,sky_is_clear
43578,10,62,1020,12,3,sky_is_clear


In [ ]:
## Data Treatment
path_of_data = "."#r"C:\Users\leand\OneDrive\Área de Trabalho\Aprendizado de Maquina"
data = pd.read_csv(f"{path_of_data}/humidity.csv", usecols= ['datetime', 'New York'])
data

In [ ]:
property = {
    #"datetime": pd.read_csv(f"{path_of_data}/humidity.csv", usecols=['datetime', 'New York']),
    "humidity": pd.read_csv(f"{path_of_data}/humidity.csv"),
    "pressure": pd.read_csv(f"{path_of_data}/pressure.csv"),
    "temperature": pd.read_csv(f"{path_of_data}/temperature.csv"),
    "wind": pd.read_csv(f"{path_of_data}/wind_speed.csv"),
    "weather": pd.read_csv(f"{path_of_data}/weather_description.csv"),
}




In [ ]:
dataset = pd.DataFrame(property.get('humidity')['datetime'])#property["wind"]["datetime"])

for key, values in property.items():
    dataset[key] = values['New York']

dataset = dataset.dropna().reset_index().drop('index', axis =1)
dataset

In [ ]:
for i,j in enumerate(dataset['weather']):
    dataset.loc[i,'weather'] = dataset.loc[i, 'weather'].replace(' ', '_')
dataset

In [7]:
dataset['temperature'] -= 273 # converte de Kevin para Celsius

In [8]:
#discretizando para aplicar Navie Bayes

dataset["humidity"] = dataset["humidity"].astype("int")
dataset["pressure"] = dataset["pressure"].astype("int")
dataset["temperature"] = dataset["temperature"].astype("int")
dataset["wind"] = dataset["wind"].astype("int")


In [10]:

#filtra o dataset para retirar outliers (no caso, valores de clima que aparecem menos que X vezes)
X = 10

count = dataset['weather'].value_counts()

to_remove_from_dataset = dataset[dataset['weather'].isin(count[count <= X].index)]  #
to_remove_from_dataset

#remove os objetos filtrados do dataset
dataset.drop(to_remove_from_dataset.index, axis = 0, inplace= True)
dataset.reset_index(inplace=True, drop=True)
dataset

,month,humidity,pressure,temperature,wind,weather
0,10,58,1012,15,7,few_clouds
1,10,57,1012,15,7,few_clouds
2,10,57,1012,15,7,few_clouds
3,10,57,1012,15,7,few_clouds
4,10,57,1012,15,6,few_clouds
...,...,...,...,...,...,...
43575,10,36,1019,16,3,sky_is_clear
43576,10,38,1019,16,1,sky_is_clear
43577,10,54,1019,14,2,sky_is_clear
43578,10,62,1020,12,3,sky_is_clear


In [20]:
mapping = {
      1 : 'summer',
      2 : 'summer',
      3 : 'autumn',
      4 : 'autumn',
      5 : 'autumn',
      6 : 'winter',
      7 : 'winter',
      8 : 'winter',
      9 : 'spring',
      10: 'spring',
      11: 'spring',
      12: 'summer',
    }

dataset['month'] = dataset['month'].map(mapping)
dataset = dataset.rename({'month': 'season'}, axis = 1)

In [21]:
dataset

,month,humidity,pressure,temperature,wind,weather
0,spring,58,1012,15,7,few_clouds
1,spring,57,1012,15,7,few_clouds
2,spring,57,1012,15,7,few_clouds
3,spring,57,1012,15,7,few_clouds
4,spring,57,1012,15,6,few_clouds
...,...,...,...,...,...,...
43575,spring,36,1019,16,3,sky_is_clear
43576,spring,38,1019,16,1,sky_is_clear
43577,spring,54,1019,14,2,sky_is_clear
43578,spring,62,1020,12,3,sky_is_clear


In [22]:
dataset.describe()

,humidity,pressure,temperature,wind
count,43580.000000,43580.000000,43580.000000,43580.000000
mean,66.651377,1017.040317,12.098417,3.221363
std,19.865275,10.142214,10.032400,2.173721
min,10.000000,959.000000,-22.000000,0.000000
25%,51.000000,1012.000000,4.000000,2.000000
50%,68.000000,1017.000000,13.000000,3.000000
75%,83.000000,1023.000000,20.000000,4.000000
max,100.000000,1052.000000,37.000000,25.000000


In [23]:
dataset.to_csv('dados_originais.csv')